# Technical Specification: Interactive Point Cloud
**RCA Exhibition — Mixed Reality Mobile Application**

**Stack:** Three.js · WebGL GLSL · MediaPipe Hands · Web Audio API · GitHub Pages

## 1. Overview

A browser-based experience rendering an animated point cloud (~200k points) from Luma AI. The cloud responds in real time to audio, touch, and visitor hand presence via camera-based hand tracking. ~66k points participate in a flocking (boids) simulation. All per-point computation is GPU-bound via custom WebGL shaders.

## 2. File Architecture

```
index.html
├── main.js                  # Scene orchestrator (Three.js)
├── loaders/plyLoader.js     # PLY → BufferGeometry
├── shaders/
│   ├── pointCloud.vert      # Vertex shader
│   └── pointCloud.frag      # Fragment shader
├── systems/
│   ├── audioSystem.js       # Web Audio API → uniforms
│   ├── handTracker.js       # MediaPipe → hand position uniforms
│   └── flockingSystem.js    # Boids → GPU texture
└── assets/scan.ply
```

## 3. The Core Split: Three.js vs. Shader

| Concern | Three.js (CPU) | Shader (GPU) |
|---|---|---|
| Load PLY, scene graph | ✓ | |
| Update uniforms per frame | ✓ | |
| Audio FFT analysis | ✓ | |
| Hand position detection | ✓ | |
| Boids neighbour lookup | ✓ (→ texture) | |
| Per-point position update | | ✓ |
| Per-point hand repulsion | | ✓ |
| Per-point glow intensity | | ✓ |
| Audio-driven displacement | | ✓ |
| Point size & colour | | ✓ |

## 4. Shader Uniforms Interface

What Three.js passes to the GPU every frame:

```glsl
uniform float u_time;
uniform float u_audioBass;      // 0.0–1.0
uniform float u_audioMid;
uniform float u_audioHigh;
uniform vec3  u_handPos[2];     // World-space hand positions
uniform float u_handPresence;   // 0.0–1.0
uniform float u_handRadius;     // Influence radius
uniform sampler2D u_flockTex;   // Encoded boids velocities
uniform vec2  u_touchPos;
uniform float u_touchActive;
uniform float u_pointSize;
uniform float u_glowIntensity;
```

## 5. Vertex Shader Logic (per-point, ~200k parallel)

```
1. Read base position from PLY geometry attribute
2. If flocking point: sample u_flockTex, apply boids velocity
3. Calculate distance to each hand
4. If within u_handRadius: apply repulsion vector scaled by proximity
5. Audio displacement:
   Bass  → radial expansion
   Mid   → noise shimmer
   High  → vertical flutter
6. Compute glow varying = 1 - (distToHand / u_handRadius)
7. Output gl_Position + gl_PointSize (scaled by audio + depth)
```

## 6. Fragment Shader Logic

```
1. Discard corners → circular points
2. Base colour from PLY vertex colour
3. Glow: mix(baseColour, glowColour, vGlow * u_glowIntensity)
4. Depth-based alpha fade
```

## 7. Flocking System

- 66k points flagged via vertex attribute `a_isFlock`
- Boids runs on CPU on a **proxy of max 5k agents** (performance)
- Velocities encoded into a float texture → `u_flockTex`
- Vertex shader reads texture, adds velocity to base position

## 8. Hand Tracking System

- Library: **MediaPipe Hands** (CDN, fully browser-side)
- Detects up to 2 hands, 21 landmarks each
- Key landmarks: wrist (0) + index fingertip (8)
- Convert normalised MediaPipe coords → Three.js world space via camera unproject
- Glow: `smoothstep(u_handRadius, 0.0, dist)` in fragment shader for soft falloff

## 9. Performance Targets

| Metric | Target |
|---|---|
| Frame rate | 60fps desktop / 30fps mobile |
| Total points | 200,000 |
| Flock points | ~66,000 |
| Boids proxy agents | ≤ 5,000 |
| Hand tracking latency | < 50ms |

## 10. Open Questions

1. Should flocking affect all 200k points or stay at 66k?
2. Target device — desktop browser or mobile?
3. Audio source — live microphone or pre-loaded track?
4. Body presence detection needed, or hands only?
5. ShaderPark — prototyping only, then port to raw GLSL for production?